This was my shot at writing few common layers used in Deep Learnning. Most of the efficient implementations are already present in the Keras API

In [1]:
import tensorflow as tf

#### A basic RNN layers. This is just for fun and not designed to be used in live deep learning models

In [36]:

class RNNCell(tf.keras.layers.Layer):
    # to be used in RNNLayer
    def __init__(self, hidden_dimension, num_classes=2, name=None):
        super(RNNCell, self).__init__(name=name)
        if hidden_dimension is None:
            raise ValueError("hidden_dimension cannot be None")
        self.hidden_dimension = hidden_dimension
        self.num_classes = num_classes

    def build(self, input_shape):
        self.w_one = tf.Variable(
            tf.random.normal([input_shape[-1], self.hidden_dimension]), name='w_one')
        self.w_two = tf.Variable(
            tf.random.normal([self.hidden_dimension, self.hidden_dimension]), name='w_two')

        self.b = tf.Variable(tf.zeros([self.hidden_dimension]), name='b')

        self.v = tf.Variable(
            tf.random.uniform([self.hidden_dimension, self.num_classes]), name='v')
        self.c = tf.Variable(tf.zeros([self.num_classes]), name='c')

    def call(self, inputs, states):
        # input shape --> (batch size, embed dimension)

        at = tf.linalg.matmul(inputs, self.w_one) + tf.linalg.matmul(states, self.w_two) + self.b
        state = tf.nn.tanh(at)
        ot = self.c + tf.linalg.matmul(state, self.v)
        yt = tf.nn.softmax(ot)
        return yt, state


class RNNLayer(tf.keras.layers.Layer):
    def __init__(self, hidden_dimension, num_classes=2, name=None):
        super(RNNLayer, self).__init__(name=name)
        self.hidden_dimension = hidden_dimension
        self.num_classes = num_classes
        self.rnn_cell = RNNCell(
            hidden_dimension=self.hidden_dimension,
            num_classes=self.num_classes
        )

    def build(self, input_shape):
        pass

    def call(self, inputs):
        time_first_input = tf.transpose(inputs, perm=[1, 0, 2])  # steps * batch * embedding

        current_state = tf.zeros(shape=(time_first_input.shape[1], self.hidden_dimension))

        states = []
        outputs = []
        # TODO void doing lists
        for i in range(time_first_input.shape[0]):
            yt, current_state = self.rnn_cell(inputs=time_first_input[i], states=current_state)
            states.append(current_state)
            outputs.append(yt)
        outputs = tf.stack(outputs)
        states = tf.stack(states)
        outputs = tf.transpose(outputs, perm=[1, 0, 2])
        states = tf.transpose(states, perm=[1, 0, 2])
        return outputs, states

In [37]:
rnn_layer = RNNLayer(hidden_dimension=100, num_classes=2)

In [38]:
inputs = tf.random.uniform(shape=(32,10, 50))

In [39]:
outputs, states = rnn_layer(inputs)

### Verify softmax output sums. They should very close to one.

In [48]:
tf.reduce_sum(outputs[0], axis=-1)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([1.        , 1.        , 0.99999994, 1.        , 0.99999994,
       1.        , 1.        , 1.        , 1.        , 0.99999994],
      dtype=float32)>

#### Quick note: Guess why some outputs are not exactly 1.0

In [49]:
outputs.__len__()

32

In [50]:
outputs[0].shape

TensorShape([10, 2])

In [51]:
states.shape

TensorShape([32, 10, 100])